In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For data visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns; sns.set()
# Plotly for interactive graphics 
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

import missingno as msno
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tdf = pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv').sample(10000)
tdf.head()

In [ ]:
tdf.info()

In [ ]:
tdf.shape

In [ ]:
tdf.isnull().sum()

In [ ]:
import missingno as msno
msno.matrix(tdf);

In [ ]:
msno.bar(tdf);

In [ ]:
missing_val = []
missing_val_rate = []
for i in tdf.columns:
    u = tdf[i].isnull().sum()
    r = (u/len(tdf)*100)
    missing_val.append(u)
    missing_val_rate.append(r)
    
y=pd.DataFrame( {"no of missing_val": missing_val,  "missing_val_rate": missing_val_rate},  index=tdf.columns)
y.sort_values("missing_val_rate", ascending=False)

In [ ]:
pd.DataFrame({"No. of unique values": list(tdf.nunique())}, index=tdf.columns)

In [ ]:
tdf.dropna(subset=['RainTomorrow','RainToday'], inplace = True)

In [ ]:
tdf.isnull().sum()

In [ ]:
sns.countplot(x="RainToday",data=tdf)

In [ ]:
 tdf.info()

In [ ]:
print(tdf.shape)
print(tdf.ndim)
print(tdf.size)

In [ ]:
unique_val = []
for i in tdf.select_dtypes(include=['object']).columns:
    u = tdf[i].nunique()
    unique_val.append(u)
    
pd.DataFrame({"No. of unique values": unique_val}, index=tdf.select_dtypes(include=['object']).columns)

In [ ]:
tdf_multi = tdf.copy()
tdf_multi = tdf_multi.drop(['Location','WindGustDir','WindDir9am','WindDir3pm','Date'], axis=1)
tdf_multi = pd.get_dummies(tdf_multi, drop_first=True, columns = ['RainToday','RainTomorrow'], prefix = ['RainToday','RainTomorrow'])
tdf_multi.head()

In [ ]:
tdf.isnull().sum()

In [ ]:
# DecisionTreeRegressor

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor

index = tdf_multi.index
columns = tdf_multi.columns

imp_mean = IterativeImputer(random_state=0, estimator=DecisionTreeRegressor())
imp_mean.fit(tdf_multi)
tdf_multi_imputed = imp_mean.transform(tdf_multi)
tdf_multi_imputed

In [ ]:
tdf_multi_imputed = pd.DataFrame(tdf_multi_imputed, index=index, columns=columns)
tdf_multi_imputed.isnull().sum()

In [ ]:
tdf_multi_imputed.info()

In [ ]:
tdf_multi_imputed.isnull().sum()

In [ ]:
sns.countplot(x="RainTomorrow_Yes",data=tdf_multi_imputed)

# PREDICTION WITH CLASSIFICATION METHODS
# Preparation dependent and independent variables

In [ ]:
tdf2 = tdf_multi_imputed.copy()
x_dat = tdf2.drop(['RainTomorrow_Yes'],axis=1)
y = tdf2["RainTomorrow_Yes"].values

# Normalization of variables

In [ ]:
#If there is a outlier values, it must be done before coming here
x=(x_dat-np.min(x_dat))/(np.max(x_dat)-np.min(x_dat)).values  

# Test-Train splitting

In [ ]:

from sklearn.model_selection import train_test_split,cross_val_score,ShuffleSplit,GridSearchCV
from sklearn.metrics import accuracy_score,mean_squared_error,roc_curve,roc_auc_score,classification_report,r2_score,confusion_matrix

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

# 1) logistic regression
# A ) Modeling of Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model=LogisticRegression() #default olanlar gelir.C var..
lr_model.fit(x_train,y_train)

In [ ]:
print(lr_model.intercept_)
print(lr_model.coef_)#Bu katsayilar denklemin katsayilari(ax+b.. gibi). mesela (-) olanlar ters yonde etkiliyor.

# B ) Prediction of Logistic Regression

In [ ]:
y_pred=lr_model.predict(x_test)

In [ ]:
y_pred[0:10]#tahminlerin ilk 10 degerini gorduk

# C ) Accuracy Test(for default) of Logistic regression

In [ ]:
accuracy_score(y_test,y_pred)

# Proba values - probability¶

In [ ]:
y_probs = lr_model.predict_proba(x_test)[:,1]

In [ ]:
y_probs

In [ ]:

y_pred = [1 if i >0.65 else 0 for i in y_probs]
y_pred[:10]

In [ ]:
log_score = accuracy_score(y_test,y_pred)
print ("log score=",log_score)

In [ ]:
confusion_matrix(y_test,y_pred)

# D ) Model tuning of Logistic regression

In [ ]:
lr_model = LogisticRegression(solver = "liblinear")
lr_model= lr_model.fit(x_train,y_train)
lr_model

In [ ]:
lr_model.predict(x_test)

In [ ]:
accuracy_score(y_test, lr_model.predict(x_test))

In [ ]:
#Cross validation (10 katli ) yaparsak
lr_finalscore=cross_val_score(lr_model, x_test, y_test, cv = 10).mean()
lr_finalscore


# 2) Modeling of Gaussian Naive Bayes
  # A ) Modeling of NB


In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model = nb_model.fit(x_train, y_train)
nb_model

# B ) Prediction of NB

In [ ]:
y_pred = nb_model.predict(x_test)
y_pred

# C ) Accuracy score of NB

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

# D ) Model Tuning of NB

In [ ]:
# 10 katli cross validation.
nb_finalscore=cross_val_score(nb_model, x_test, y_test, cv = 10).mean()
nb_finalscore

# 3 ) KNN
  # A ) Modeling of KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()
knn_model = knn_model.fit(x_train, y_train)
knn_model

# B ) Prediction of KNN

In [ ]:
y_pred = knn_model.predict(x_test)

# C ) Accuracy score of KNN

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test,y_pred)

# D ) Model Tuning of KNN

In [ ]:
knn_params = {"n_neighbors": np.arange(1,50)}

In [ ]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv=10)
knn_cv.fit(x_train, y_train)

In [ ]:
print("The best score:" + str(knn_cv.best_score_))
print("The best parameters: " + str(knn_cv.best_params_))

In [ ]:
knn = KNeighborsClassifier(3)  #we choose 3 neigboors. I think 1 is not good 
knn_tuned = knn.fit(x_train, y_train)

In [ ]:
knn_finalscore=knn_tuned.score(x_test, y_test)
knn_finalscore

# 4 )SVC
  # A ) Modeling of SVC

In [ ]:
from sklearn.svm import SVC
svm_model = SVC().fit(x_train,y_train)#we choose default c:1,kernel:'rbf',dagree:3...
#?svm_model

# B ) Prediction of SVC

In [ ]:
y_pred = svm_model.predict(x_test)

# C ) Accuracy score of SVC

In [ ]:
accuracy_score(y_test,y_pred)

# D ) Model Tuning of SVC
   # For kernel:rbf

In [ ]:
svc_params = {"C": np.arange(1,10)}

svc = SVC(kernel = "rbf")

svc_cv_model = GridSearchCV(svc,svc_params, 
                            cv = 10, 
                            n_jobs = -1,        
                            verbose = 2 )

svc_cv_model.fit(x_train, y_train)

In [ ]:
print("The best parameters: " + str(svc_cv_model.best_params_))

In [ ]:
svc_tuned1 = SVC(kernel = "rbf", C = 6).fit(x_train, y_train)

y_pred = svc_tuned1.predict(x_test)
accuracy_score(y_test, y_pred)

# For kernel:linear

In [ ]:
svc_params = {"C": np.arange(1,10)}

svc = SVC(kernel = "linear")

svc_cv_model = GridSearchCV(svc,svc_params, 
                            cv = 10, 
                            n_jobs = -1, 
                            verbose = 2 )

svc_cv_model.fit(x_train, y_train)

In [ ]:
print("The best parameters: " + str(svc_cv_model.best_params_))

In [ ]:
svc_tuned2 = SVC(kernel = "linear", C = 9).fit(x_train, y_train)
y_pred = svc_tuned2.predict(x_test)
accuracy_score(y_test, y_pred)

# For kernel:rbf , C and gamma

In [ ]:
svc_model = SVC(kernel = "rbf").fit(x_train, y_train)

In [ ]:
svc_params = {"C": [0.0001, 0.001, 0.1, 1, 5, 10 ,50 ,100],
             "gamma": [0.0001, 0.001, 0.1, 1, 5, 10 ,50 ,100]}

In [ ]:
svc = SVC()
svc_cv_model = GridSearchCV(svc, svc_params, 
                         cv = 10, 
                         n_jobs = -1,
                         verbose = 2)

svc_cv_model.fit(x_train, y_train)

In [ ]:
print("The best parameters: " + str(svc_cv_model.best_params_))

In [ ]:
svc_tuned3 = SVC(C = 100, gamma =0.001).fit(x_train, y_train)
y_pred = svc_tuned3.predict(x_test)
svc_finalscore=accuracy_score(y_test, y_pred)
svc_finalscore

# 5) Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier()
rf_model.fit(x_train,y_train)

# Prediction of Random Forests

In [ ]:
y_pred = rf_model.predict(x_test)

# Accuracy score of Random Forests

In [ ]:
rf_score=accuracy_score(y_test,y_pred)
rf_score

In [ ]:
c_rf=confusion_matrix(y_test,y_pred)
c_rf

In [ ]:
from sklearn.metrics import confusion_matrix   #Hepsi icin yapilabilir
y_true=y_test
y_pred=rf_model.predict(x_test)
cmlr=confusion_matrix(y_true, y_pred)
f,ax=plt.subplots(figsize=(6,6))
sns.heatmap(cmlr, annot=True)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()

# CLASSIFICATION REPORT: 

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
rf_model.predict(x_test)[0:10] #ilk 10 datatest deki tahminlerimiz

In [ ]:
rf_model.predict_proba(x_test)[0:10] #1.si 0 olma 2.si 1 olma olasiligi oranlari


# TUNING THE PREDICTION: WE can tune our prediction
n_estimators,importance variables,

In [ ]:
from sklearn.ensemble import RandomForestClassifier  #n_estimotors=11 is best
score_list=[]
for each in range(1,75):
    rf2=RandomForestClassifier(n_estimators=each, random_state=42)
    rf2.fit(x_train, y_train)
    score_list.append(100*rf2.score(x_test, y_test))
    print("n_estimators=", each, "--> Accuracy:", 100*rf2.score(x_test, y_test), "%")

plt.plot([*range(1,75)], score_list)
plt.xlabel("n_estimators Value")
plt.ylabel("Accuracy %")
plt.show()

# Lets look at importance of variables

In [ ]:
Importance = pd.DataFrame({"Importance": rf_model.feature_importances_*100},
                         index = x_train.columns)
Importance.sort_values(by = "Importance", 
                       axis = 0, 
                       ascending = True).plot(kind ="barh", color = "g")

plt.xlabel("Variable Importance Levels");

In [ ]:
y_pred = rf2.predict(x_test)
rf_finalscore=accuracy_score(y_test, y_pred)
rf_finalscore

# 6) DECISION TREE METHOD

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)

In [ ]:
y_pred = dtc.predict(x_test)
y_pred

In [ ]:
dtc_score=accuracy_score(y_test,y_pred)
dtc_score

In [ ]:
c_dtc=confusion_matrix(y_test,y_pred)
c_dtc

In [ ]:
from sklearn.metrics import confusion_matrix   #Hepsi icin yapilabilir
y_true=y_test
y_pred=dtc.predict(x_test)
cmdtc=confusion_matrix(y_true, y_pred)
f,ax=plt.subplots(figsize=(6,6))
sns.heatmap(cmdtc, annot=True)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))

# Decision Tree Model tuning

In [ ]:
tree_grid = {"max_depth": range(1,10),
            "min_samples_split" : list(range(2,50)) }

In [ ]:
tree1 = DecisionTreeClassifier()
tree_cv = GridSearchCV(tree1, tree_grid, cv = 10, n_jobs = -1, verbose = 2)
tree_cv_model = tree_cv.fit(x_train, y_train)

In [ ]:
print("Best Parameters: " + str(tree_cv_model.best_params_))

In [ ]:
tree1 = DecisionTreeClassifier(max_depth = 5, min_samples_split = 23)
tree_tuned1 = tree1.fit(x_train, y_train)

In [ ]:
y_pred = tree_tuned1.predict(x_test)
dtc_finalscore=accuracy_score(y_test, y_pred)
dtc_finalscore

In [ ]:
c_dtc2=confusion_matrix(y_test,y_pred)
c_dtc2

In [ ]:
print(classification_report(y_test,y_pred))

# 7) Gradient Boosting Machines

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbm_model = GradientBoostingClassifier().fit(x_train, y_train)

In [ ]:
y_pred = gbm_model.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

- Model Tuning of GBM

In [ ]:
gbm = GradientBoostingClassifier(learning_rate = 0.05, 
                                 max_depth = 10,
                                min_samples_split = 10,
                                n_estimators = 100)

In [ ]:
gbm_tuned =  gbm.fit(x_train,y_train)

In [ ]:
y_pred = gbm_tuned.predict(x_test)
gbm_finalscore=accuracy_score(y_test,y_pred)
gbm_finalscore

In [ ]:
c_gbm=confusion_matrix(y_test,y_pred)
c_gbm

In [ ]:
print(classification_report(y_test,y_pred))

# 8 ) XGBoost

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier().fit(x_train, y_train)

In [ ]:
y_pred = xgb_model.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

- Model Tuning of XGBoost

In [ ]:
xgb = XGBClassifier(learning_rate = 0.01, 
                    max_depth = 6,
                    n_estimators = 100,
                    subsample = 0.8)
xgb_tuned =  xgb.fit(x_train,y_train)
y_pred = xgb_tuned.predict(x_test)
XGBoost_finalscore=accuracy_score(y_test, y_pred)
XGBoost_finalscore

In [ ]:
c_xgb=confusion_matrix(y_test,y_pred)
c_xgb

In [ ]:
print(classification_report(y_test,y_pred))

# 9 ) LightGBM

In [ ]:
!conda install -c conda-forge lightgbm

In [ ]:
from lightgbm import LGBMClassifier
lgbm_model = LGBMClassifier().fit(x_train, y_train)

In [ ]:
y_pred = lgbm_model.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

- Model Tuning of LightGBM

In [ ]:
lgbm_params = {
        'n_estimators': [100, 500, 1000, 2000],
        'subsample': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5,6],
        'learning_rate': [0.1,0.01,0.02,0.05],
        "min_child_samples": [5,10,20]}

In [ ]:
lgbm = LGBMClassifier(learning_rate = 0.1, 
                       max_depth = 4,
                       subsample = 0.6,
                       n_estimators = 500,
                       min_child_samples = 10)
lgbm_tuned = lgbm.fit(x_train,y_train)

In [ ]:
y_pred = lgbm_tuned.predict(x_test)
lgbm_finalscore=accuracy_score(y_test, y_pred)
lgbm_finalscore

In [ ]:
c_lgbm=confusion_matrix(y_test,y_pred)
c_lgbm

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print(lr_finalscore,nb_finalscore,knn_finalscore,svc_finalscore, rf_finalscore, dtc_finalscore, gbm_finalscore, XGBoost_finalscore, lgbm_finalscore)

In [ ]:

indexx = ["Log","NB","KNN","SVC", "RF", "DT", "GBM", "XGB", "LGBM"]
regressions = [lr_finalscore,nb_finalscore,knn_finalscore,svc_finalscore, rf_finalscore, dtc_finalscore, gbm_finalscore, XGBoost_finalscore, lgbm_finalscore]

plt.figure(figsize=(8,6))
sns.barplot(x=indexx,y=regressions)
plt.xticks()
plt.title('Model Comparision',color = 'orange',fontsize=20);

